<a href="https://colab.research.google.com/github/ravi-gopalan/DAND_Data_Wrangling/blob/master/review_processing_2020_02_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
cd '/gdrive/My Drive/abv_reviews'

/gdrive/My Drive/abv_reviews


In [3]:
!pip install -U spacy[cuda100]
!pip install textdistance
!pip install "textdistance[extras]"

     |████████████████████████████████| 10.4MB 76kB/s 
     |████████████████████████████████| 2.2MB 70.1MB/s 
     |████████████████████████████████| 3.7MB 54.6MB/s 
     |████████████████████████████████| 122kB 74.7MB/s 
     |████████████████████████████████| 382.9MB 42kB/s 
  Found existing installation: preshed 2.0.1
    Uninstalling preshed-2.0.1:
      Successfully uninstalled preshed-2.0.1
  Found existing installation: blis 0.2.4
    Uninstalling blis-0.2.4:
      Successfully uninstalled blis-0.2.4
  Found existing installation: thinc 7.0.8
    Uninstalling thinc-7.0.8:
      Successfully uninstalled thinc-7.0.8
  Found existing installation: spacy 2.1.9
    Uninstalling spacy-2.1.9:
      Successfully uninstalled spacy-2.1.9
     |████████████████████████████████| 61kB 3.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 12.5MB/s 
     |████████████

In [4]:
import spacy.cli
spacy.cli.download("en_core_web_lg")
import en_core_web_lg

⚠ Skipping model package dependencies and setting `--no-deps`. You
don't seem to have the spaCy package itself installed (maybe because you've
built from source?), so installing the model dependencies would cause spaCy to
be downloaded, which probably isn't what you want. If the model package has
other dependencies, you'll have to install them manually.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import re
import random
import srsly
import json
import spacy
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
from spacy import displacy
from spacy.util import minibatch, compounding, decaying
from pandas.io.json import json_normalize
from spacy.gold import docs_to_json, biluo_tags_from_offsets, offsets_from_biluo_tags
from spacy.matcher import Matcher 
from spacy.tokens import Span

import itertools
from collections import Counter

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import textdistance as td
import jellyfish as jf


In [6]:
spacy.prefer_gpu()

True

In [0]:
def cleanse_text(text):
  step1 = re.sub(r"(\{'LOWER':\s)",r"",str(text))
  step2 = re.sub(r"[\[{\'\,\"}\]]",r"",step1)
  return step2

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']

#        print(text, annotations)
        valid_entities = []
        for start, end, label in entities:
          valid_start = start
          valid_end = end
#          print(valid_start, valid_end, len(text))
          while valid_start < len(text) and invalid_span_tokens.match(text[valid_start]):
            valid_start += 1
          while valid_end > 1 and invalid_span_tokens.match(text[valid_end - 1]):
            valid_end -= 1
          valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data

def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')
    print('Wrote {} records to {}'.format(len(data), output_path))

def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data


def create_entity_pattern(ent_name, ent_label):
  pattern_list = []
  pattern_dict = {}
  id_str = ''
  for word in ent_name.lower().split():
    token_dict = {}
    token_dict['LOWER'] = word
    pattern_list.append(token_dict)
    id_str += '_' + word
  if ent_label[:2] == 'U_':
    pattern_dict['label'] = ent_label[2:]
  else:
    pattern_dict['label'] = ent_label  
  pattern_dict['pattern'] = pattern_list
  if id_str[0] == '_':
    id_str = id_str[1:]
  pattern_dict['id'] = id_str
  return pattern_dict


def clean_like_objects(text):
  step1 = re.sub("ObjectId\(\'","",str(text))
  step2 = re.sub("\'\)","",step1)
  return step2


In [0]:
!cat beverage.jsonl bread.jsonl card_date_event_fac.jsonl > test1.jsonl
!cat condiment.jsonl dish.jsonl event_food_habit.jsonl > test2.jsonl
!cat fruit.jsonl herb_lifestyle_meal.jsonl loc_money_norp_ordinal.jsonl > test3.jsonl
!cat org_product.jsonl pastry_seed_spice.jsonl quant_time.jsonl > test4.jsonl

!cat test1.jsonl test2.jsonl test3.jsonl test4.jsonl > test5.jsonl
!cat test5.jsonl updated_GPE.jsonl partner.jsonl vegetable.jsonl > patterns_2020_02_08.jsonl

In [8]:
df_pattern = pd.read_json('patterns_2020_02_08.jsonl',lines=True)
df_pattern['cleaned_pattern'] = df_pattern.pattern.apply(lambda x: cleanse_text(x))
df_pattern['label'] = 'U_' + df_pattern.label.astype(str)
df_pattern['tup_col'] = list(zip(df_pattern.cleaned_pattern, df_pattern.label))
df_pattern['merged'] = df_pattern.apply(lambda row: {row['cleaned_pattern']:row['label']}, axis=1)
df_pattern.head()
df_pattern.tail()

,label,pattern,id,cleaned_pattern,tup_col,merged
22511,U_vegetable,[{'LOWER': 'zucchini'}],zucchini,zucchini,"(zucchini, U_vegetable)",{'zucchini': 'U_vegetable'}
22512,U_vegetable,[{'LOWER': 'zucchinis'}],zucchini,zucchinis,"(zucchinis, U_vegetable)",{'zucchinis': 'U_vegetable'}
22513,U_vegetable,[{'LOWER': 'zuccini'}],zucchini,zuccini,"(zuccini, U_vegetable)",{'zuccini': 'U_vegetable'}
22514,U_vegetable,[{'LOWER': 'zuchini'}],zucchini,zuchini,"(zuchini, U_vegetable)",{'zuchini': 'U_vegetable'}
22515,U_vegetable,[{'LOWER': 'zuchinni'}],zucchini,zuchinni,"(zuchinni, U_vegetable)",{'zuchinni': 'U_vegetable'}


In [0]:
ent_list = df_pattern.tup_col.to_list()

In [0]:
nlp = en_core_web_lg.load()
ruler = EntityRuler(nlp).from_disk("patterns_2020_02_08.jsonl")
nlp.add_pipe(ruler, before='ner')
merge_ents = nlp.create_pipe("merge_entities")
nlp.add_pipe(merge_ents, after='ner')
nlp.pipe_names

In [10]:
reviews = pd.read_csv('reviews.csv')
reviews['createdAt'] = pd.to_datetime(reviews['createdAt'],format='%Y-%m-%d %H:%M:%S.%f')
reviews['text'] = reviews['text'].astype(str)
reviews.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159795 entries, 0 to 159794
Data columns (total 47 columns):
_id                                       159795 non-null object
status                                    159795 non-null object
likes                                     159795 non-null object
likesCount                                159795 non-null int64
commentsCount                             159794 non-null float64
user                                      159795 non-null object
text                                      159795 non-null object
type                                      159795 non-null object
taste                                     107028 non-null float64
presentation                              107028 non-null float64
nutrition                                 107028 non-null float64
originality                               107028 non-null float64
value                                     107028 non-null float64
overall                                

In [0]:
train_reviews = reviews[:3000]
eval_reviews = reviews[3000:133000]
test_reviews = reviews[133000:]

In [0]:
train_gen = (rev for rev in train_reviews['text'])
to_train_ents = []
check_list = []

# Load the NLP Model

In [0]:
nlp = spacy.load('trained_model_2020_02_08')

In [13]:
ent_list_eval = []
for doc in nlp.pipe(eval_reviews['text']):
#  displacy.render(nlp(doc),'ent',jupyter=True)
  ent_list_eval.append([k  for k in Counter(sorted([(ent.ent_id_) for ent in doc.ents if ent.ent_id_ != ''])).keys()])
#ent_list.append([ent.text for ent in nlp(doc).ents if ent.ent_id_ == ''])

ent_list_eval[-5:]

[['vegan'],
 ['green_tea', 'tea'],
 ['el_hogar_animal_sanctuary'],
 ['tea'],
 ['lentil']]

In [14]:
ent_list_eval[:5]

[['vegan'],
 ['garbanzo', 'hamburger'],
 ['basil', 'cashew_nuts', 'cherry_tomatoes', 'pasta', 'tea'],
 [],
 ['milanesa']]

In [15]:
df_entities = pd.DataFrame(pd.Series(ent_list_eval),columns=['entities'])
df_entities.to_csv('entities_3_133k.csv',index=False)
df_entities.tail()

,entities
129995,[vegan]
129996,"[green_tea, tea]"
129997,[el_hogar_animal_sanctuary]
129998,[tea]
129999,[lentil]


In [16]:
eval_reviews.reset_index(drop=True)

,_id,status,likes,likesCount,commentsCount,user,text,type,taste,presentation,nutrition,originality,value,overall,createdAt,dish_info.isActive,dish_info.name,dish_info.description,restaurant_info.name,restaurant_info.country,restaurant_info.website,images_info.path,images_info.public_id,user_info.name,user_info.reviewsCount,user_info.likedArticlesCount,user_info.followersCount,user_info.likedReviewsCount,user_info.challengeReviews,user_info.username,user_info.language,user_info.home,user_info.reviewKarma.approved.count,user_info.reviewKarma.approved.score,user_info.reviewKarma.ineligible.count,user_info.reviewKarma.ineligible.score,user_info.reviewKarma.total.count,user_info.reviewKarma.total.score,restaurant_info.locality,isProduct,product_info.brand,product_info.brandCategory,product_info.name,product_info.normalizedName,product_info.brandName,product_info.category,product_info.subcategory
0,5e2f312ee873b10020b64c5d,approved,"[ObjectId('5e03e8307956290020e51e8d'), ObjectI...",2,0.0,5e09716a3a202700209d8f8d,Mitad cancha mitad super cancha (con queso veg...,dish,5.0,5.0,5.0,5.0,5.0,5.0,2020-01-27 18:51:26.422,True,Pizza con paparella,NaN,Pizzas y Empanadas IL Giardino De Devoto,Argentina,https://www.facebook.com/ilgiardinodidevoto/,https://res.cloudinary.com/abillionveg/image/u...,zuex5u2cdlw7xsivvdnh,germancampagno,43,0,11,19,13,germancampagno,en,Argentina,40,40,0,0,43,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5e2f30cd21cbcd002090abdd,approved,"[ObjectId('5e03e8307956290020e51e8d'), ObjectI...",2,0.0,5def888d287cf90020c983dc,Hamburguesa de porotos o de garbanzos con ques...,dish,5.0,5.0,4.0,4.0,4.0,4.4,2020-01-27 18:49:49.544,True,Hamburguesa cheddar,NaN,Rincón Flamenco Café,Chile,NaN,https://res.cloudinary.com/abillionveg/image/u...,zzgyjlrzxqamenxsmr2h,Estefa,7,0,1,1,7,wicca,es,Chile,7,7,0,0,7,7,La Serena,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5e2f30c521cbcd002090abd4,approved,"[ObjectId('5e03e8307956290020e51e8d'), ObjectI...",2,0.0,5e1a06a6ce56f60020488a73,Creamy and elegant pasta with cherry tomatoes ...,dish,5.0,5.0,5.0,4.0,5.0,4.8,2020-01-27 18:49:41.422,True,Pasta Alfredo,NaN,Veda House,Bulgaria,http://vedahouse.business.site/,https://res.cloudinary.com/abillionveg/image/u...,n94zmgxh9srfgmj2tpes,anitagmer,8,0,2,0,8,anitagmer,es,Spain,8,8,0,0,8,8,Sofia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5e2f305d21cbcd002090abaf,approved,"[ObjectId('5e03e8307956290020e51e8d'), ObjectI...",2,0.0,5e09716a3a202700209d8f8d,Despues del green doner es lo mas rico que pro...,dish,5.0,5.0,3.0,5.0,3.0,4.2,2020-01-27 18:47:57.080,True,Green Steak,NaN,GreenFactory,Argentina,http://www.greenfactory.com.ar/,https://res.cloudinary.com/abillionveg/image/u...,qwh1ghw6uakzpfcgwb0t,germancampagno,43,0,11,19,13,germancampagno,en,Argentina,40,40,0,0,43,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5e2f3021e873b10020b64c1a,approved,"[ObjectId('5e03e8307956290020e51e8d'), ObjectI...",2,0.0,5e09716a3a202700209d8f8d,"Jamon queso huevo milanesa, de todo 🤯🤯🤯🤯🤯🤯🤯",dish,5.0,5.0,3.0,5.0,5.0,4.6,2020-01-27 18:46:57.667,True,Lomito completo,NaN,Loving Hut Microcentro,Argentina,https://www.facebook.com/LovingHut.Microcentro/,https://res.cloudinary.com/abillionveg/image/u...,mrpnftdvto4hpowgqhuh,germancampagno,43,0,11,19,13,germancampagno,en,Argentina,40,40,0,0,43,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129995,5db33a5abec7270020cc07e0,approved,"[ObjectId('5d75cb19e6d9f5000489c83c'), ObjectI...",2,0.0,5ced3ac86baff40004259115,This is vegan and quite good. You can find it ...,product,NaN,NaN,NaN,NaN,NaN,4.8,2019-10-25 18:09:30.836,NaN,NaN,NaN,NaN,NaN,NaN,https://res.cloudinary.com/abillionveg/image/u...,ipk1cysnb7gdw1ezztv0,basia,106,0,15,56,6,basia,fr,Canada,101,101,0,0,106,101,NaN,NaN,5db33a5abec7270020cc07de,food,Tranches de melon,tranches de melon,nos favoris,snacks,sweets & candies
129996,5

In [17]:
eval_new_df = pd.concat([eval_reviews.reset_index(drop=True), df_entities],axis=1,ignore_index=True)
eval_new_df.columns = list(eval_reviews.columns) + list(df_entities.columns)
eval_new_df.head()

,_id,status,likes,likesCount,commentsCount,user,text,type,taste,presentation,nutrition,originality,value,overall,createdAt,dish_info.isActive,dish_info.name,dish_info.description,restaurant_info.name,restaurant_info.country,restaurant_info.website,images_info.path,images_info.public_id,user_info.name,user_info.reviewsCount,user_info.likedArticlesCount,user_info.followersCount,user_info.likedReviewsCount,user_info.challengeReviews,user_info.username,user_info.language,user_info.home,user_info.reviewKarma.approved.count,user_info.reviewKarma.approved.score,user_info.reviewKarma.ineligible.count,user_info.reviewKarma.ineligible.score,user_info.reviewKarma.total.count,user_info.reviewKarma.total.score,restaurant_info.locality,isProduct,product_info.brand,product_info.brandCategory,product_info.name,product_info.normalizedName,product_info.brandName,product_info.category,product_info.subcategory,entities
0,5e2f312ee873b10020b64c5d,approved,"[ObjectId('5e03e8307956290020e51e8d'), ObjectI...",2,0.0,5e09716a3a202700209d8f8d,Mitad cancha mitad super cancha (con queso veg...,dish,5.0,5.0,5.0,5.0,5.0,5.0,2020-01-27 18:51:26.422,True,Pizza con paparella,NaN,Pizzas y Empanadas IL Giardino De Devoto,Argentina,https://www.facebook.com/ilgiardinodidevoto/,https://res.cloudinary.com/abillionveg/image/u...,zuex5u2cdlw7xsivvdnh,germancampagno,43,0,11,19,13,germancampagno,en,Argentina,40,40,0,0,43,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[vegan]
1,5e2f30cd21cbcd002090abdd,approved,"[ObjectId('5e03e8307956290020e51e8d'), ObjectI...",2,0.0,5def888d287cf90020c983dc,Hamburguesa de porotos o de garbanzos con ques...,dish,5.0,5.0,4.0,4.0,4.0,4.4,2020-01-27 18:49:49.544,True,Hamburguesa cheddar,NaN,Rincón Flamenco Café,Chile,NaN,https://res.cloudinary.com/abillionveg/image/u...,zzgyjlrzxqamenxsmr2h,Estefa,7,0,1,1,7,wicca,es,Chile,7,7,0,0,7,7,La Serena,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[garbanzo, hamburger]"
2,5e2f30c521cbcd002090abd4,approved,"[ObjectId('5e03e8307956290020e51e8d'), ObjectI...",2,0.0,5e1a06a6ce56f60020488a73,Creamy and elegant pasta with cherry tomatoes ...,dish,5.0,5.0,5.0,4.0,5.0,4.8,2020-01-27 18:49:41.422,True,Pasta Alfredo,NaN,Veda House,Bulgaria,http://vedahouse.business.site/,https://res.cloudinary.com/abillionveg/image/u...,n94zmgxh9srfgmj2tpes,anitagmer,8,0,2,0,8,anitagmer,es,Spain,8,8,0,0,8,8,Sofia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[basil, cashew_nuts, cherry_tomatoes, pasta, tea]"
3,5e2f305d21cbcd002090abaf,approved,"[ObjectId('5e03e8307956290020e51e8d'), ObjectI...",2,0.0,5e09716a3a202700209d8f8d,Despues del green doner es lo mas rico que pro...,dish,5.0,5.0,3.0,5.0,3.0,4.2,2020-01-27 18:47:57.080,True,Green Steak,NaN,GreenFactory,Argentina,http://www.greenfactory.com.ar/,https://res.cloudinary.com/abillionveg/image/u...,qwh1ghw6uakzpfcgwb0t,germancampagno,43,0,11,19,13,germancampagno,en,Argentina,40,40,0,0,43,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
4,5e2f3021e873b10020b64c1a,approved,"[ObjectId('5e03e8307956290020e51e8d'), ObjectI...",2,0.0,5e09716a3a202700209d8f8d,"Jamon queso huevo milanesa, de todo 🤯🤯🤯🤯🤯🤯🤯",dish,5.0,5.0,3.0,5.0,5.0,4.6,2020-01-27 18:46:57.667,True,Lomito completo,NaN,Loving Hut Microcentro,Argentina,https://www.facebook.com/LovingHut.Microcentro/,https://res.cloudinary.com/abillionveg/image/u...,mrpnftdvto4hpowgqhuh,germancampagno,43,0,11,19,13,germancampagno,en,Argentina,40,40,0,0,43,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[milanesa]


In [1]:
ent_list_eval_tba = []
for doc in nlp.pipe(eval_reviews['text']):
  ent_list_eval_tba.append([ent.text for ent in doc.ents if ent.ent_id_ == ''])

df_eval = pd.DataFrame()
df_eval = df_eval.append(ent_list_eval_tba).dropna(how='all')

full_list_eval_tba = []

for col in df_eval.columns:
  full_list_eval_tba.extend([k.lower() for k in Counter(list(df_eval[col])).keys() if k != None])

full_list_eval_tba = [k for k in Counter(sorted(full_list_eval_tba)).keys()]
len(full_list_eval_tba)

NameError: ignored

In [0]:
ent_list_test_tba = []
for doc in nlp.pipe(test_reviews['text']):
  ent_list_test_tba.append([ent.text for ent in doc.ents if ent.ent_id_ == ''])

df_test = pd.DataFrame()
df_test = df_test.append(ent_list_test_tba).dropna(how='all')

full_list_test_tba = []

for col in df_test.columns:
  full_list_test_tba.extend([k.lower() for k in Counter(list(df_test[col])).keys() if k != None])

full_list_test_tba = [k for k in Counter(sorted(full_list_test_tba)).keys()]
len(full_list_test_tba)

In [0]:
entities_to_be_added = [k for k in Counter(sorted(full_list_eval_tba + full_list_test_tba)).keys()]
len(entities_to_be_added)

In [0]:
pat_list = list(df_pattern['id'].unique())

In [0]:
leven_result = []
for inp in entities_to_be_added:

  for pat in pat_list:
    leven = td.levenshtein.normalized_similarity(inp, pat)

    if  (leven > 0.7) & (leven < 0.9999):
      leven_result.append((inp, pat, leven))
len(leven_result)

In [0]:
sound_list = [item for item in entities_to_be_added if type(item) == str]
soundex_list = []

for item in sound_list:
  try:
    soundx = jf.soundex(item)
  except:
    soundx = 'no value'
  soundex_list.append((item, soundx))

len(soundex_list)

In [0]:
dump_jsonl(entities_to_be_added,'entities_to_be_added.jsonl')
dump_jsonl(leven_result,'leven_similar.jsonl')
dump_jsonl(soundex_list,'sound_similar.jsonl')

In [0]:
eval_new_df.columns

In [0]:
df_likes = eval_new_df[['review','likes']]
df_likes['cleaned_likes'] = df_likes['likes'].apply(lambda x: clean_like_objects(x))
df_likes.head()

In [0]:
df_likes['likes'] = df_likes.cleaned_likes.apply(lambda x: x[1:-1].split(','))
df_likes.head()